# Ⅰ Web Crawling
## 1. 웹페이지 종류
- **정적** 페이지 : 페이지의 데이터가 변경될 때 **URL이 변경 O** (ex. 페이지네이션)
    - 웹 브라우져에 화면이 한번 뜨면 이벤트에 의한 화면의 변경이 없는 페이지
    - HTML 포멧의 데이터 수집
- **동적** 페이지 : 페이지의 데이터가 변경될 때 **URL이 변경 X** (ex. 더보기 버튼)
    - 웹 브라우져에 화면이 뜨고 이벤트가 발생하면 서버에서 데이터를 가져와 화면을 변경하는 페이지
    - JSON 포멧의 데이터 수집 (JSON ≒ Python의 dict)
    
## 2. requests package
- 받아오는 문자열에 따라 2 가지 방법으로 구분
    - **html 문자열**로 받아서 파싱하는 방법 : 주로 **정적 페이지** 크롤링할 때 사용
    - **json 문자열**로 받아서 파싱하는 방법 : 주로 **동적 페이지** 크롤링할 때 사용
- 브라우저의 URL을 입력하면 서버에서 데이터를 다운받아 화면에 출력 : URL -> DATA
- requests 패키지 : URL 입력 -> DATA 받아오기

## 3. selenium package
- 브라우저를 직접 열어서 데이터를 받는 방법

## 4. 크롤링 방법에 따른 속도
- requests json > requests html > selenium

-----------

## 5. Crwaling Naver Stock Datas
- 네이버 증권 사이트에서 주가 데이터 수집
- 수집할 데이터 : 일별 kospi, kosdaq 주가, 일별 환율(exchange rate) 데이터
- 데이터 수집 절차
    - 웹 서비스 분석 : url
    - 서버에 데이터 요청 : request(url) -> response : json(str)
    - 서버에서 받은 데이터 파싱(데이터 형태 변경) : json(str) -> list, dict -> DataFrame

- 요소 탭 : 화면에 대한 정보가 HTML 코드로 적혀있음
- 네트워크 탭 : 어떤 데어트를 가저오는지 알 수 있음
    - url 확인 방법 : 클린 아이콘 클릭 -> 더보기 버튼 클릭 -> '이름' 클릭 -> url 확인(미리보기 탭에 있는 데이터를 가져오는 url)

## 1) Kospi 지수

In [2]:
import requests
import pandas as pd

### ① 웹서비스 분석(url) : 크롬 개발자 도구 사용
- PC 웹페이지가 복잡하면 mobile 웹페이지에서 수집 (Ctrl + Shift + I -> Ctrl + Shift + M)
- 트래픽 발생 확인 가능

In [57]:
url = 'https://m.stock.naver.com/api/index/KOSPI/price?pageSize=10&page=8'

### ② 서버에 데이터 요청 : request(url) -> response : JSON(str)
- response의 status code가 200 인지 확인
- 403이나 500이면 request가 잘못 됐거나 web server에서 수집이 안 되도록 설정된 것
    - header 설정 또는 selenium 사용
- 200 나와도 response 안에 있는 내용 확인
    - 확인방법 : response.txt

In [75]:
# 데이터를 서버쪽에 요청(requests) 후 받은(resopnse) 데이터를 response에 저장
response = requests.get(url) # Shift + Tab하면 메소드 작성법 알 수 있음

# 200 출력되는지 확인
response 

# response 안에 내용 확인 : JSON(str) 데이터 (리스트 안에 딕셔너리)
response.text[:200]

### ③ 서버에서 받은 데이터 파싱(데이터 형태 변경)
- json(str) -> list, dict -> DataFrame

In [59]:
# JSON(str) -> list 변환 (리스트 안에 딕셔너리가 있음)
kospi_data = response.json()

# 딕셔너리 데이터가 로그 데이터가 되어 데이터프레임으로 변환
kospi = pd.DataFrame(data)[['localTradedAt', 'closePrice']]

### ④ 함수 만들기
* parmas : pasgesize, page

In [27]:
def stock_price1(pagesize, page):
    url = f'https://m.stock.naver.com/api/index/KOSPI/price?pageSize={pagesize}&page={page}'
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data)[['localTradedAt', 'closePrice']]

## 2) Kosdaq 지수

In [ ]:
#1. 웹서비스 분석 : URL
kosdaq_url = 'https://m.stock.naver.com/api/index/KOSDAQ/price?pageSize=10&page=2'

#2. request(url) -> response(json) : JSON(str)
kosdaq_response = requests.get(kosdaq_url)

#3. JSON(str) -> list, dict -> DataFrame
kosdaq_data = kosdaq_response.json()
kosdaq = pd.DataFrame(kosdaq_data)[['localTradedAt', 'closePrice']]
kosdaq.tail()

In [61]:
#4. 함수 만들기 (parmas : pasgesize, page, code)
# def stock_price(pagesize, page, code = 'KOSPI') : code값을 따로 설정하지 않으면 KOSPI로 작동
def stock_price(pagesize, page, code):
    """This function is crawling stock price from naver webpage
    
    Parmas
    ------
    pagesize : int : one page size
    page : int : page number
    code : str : KOSPI or KOSDAQ
    
    Return
    ------
    type : DataFrame : display data, price columns
    """
    
    url = f'https://m.stock.naver.com/api/index/{code}/price?pageSize={pagesize}&page={page}'
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data)[['localTradedAt', 'closePrice']]

In [43]:
#5. dostring : 함수를 사용하는 방법을 문자열로 작성
# help(), shift + tab 이용
help(stock_price)

## 3) USD : 원달러 환율
* 최근 60일치 원달러 환율 데이터 수집

In [94]:
# 몇 번째(page), 몇 개(pageSize)의 데이터를 갖는 url
page, pagesize = 1, 60
usd_url = f'https://api.stock.naver.com/marketindex/exchange/FX_USDKRW/prices?page={page}&pageSize={pagesize}'

#2. request(url) -> respone(json) : JSON(str)
usd_response = requests.get(usd_url)

#3. JSON(str) -> list, dict -> DataFrame
usd_data = usd_response.json()
usd = pd.DataFrame(usd_data)[['localTradedAt', 'closePrice']]
usd.tail(2)

## 4) EUR : 원유로 환율
* 최근 60일치 원달러 환율 데이터 수집

In [95]:
page, pagesize = 1, 60
eur_url = f'https://api.stock.naver.com/marketindex/exchange/FX_EURKRW/prices?page={page}&pageSize={pagesize}'
eur_response = requests.get(eur_url)
eur_data = eur_response.json()
eur = pd.DataFrame(eur_data)[['localTradedAt', 'closePrice']]
eur.tail(2)

-----------

## 6. 데이터 분석
## 1) 상관관계분석
- 두 데이터 집합 사이에 어떤 관계가 있는지 확인하는 분석 방법
- **피어슨 상관계수 : df.corr()**
    - 두 데이터 집합의 상관도를 분석할 때 사용되는 지표
    * 1과 가까울수록 강한 양의 상관관계
    * -1과 가까울수록 강한 음의 상관관계
    * 0과 가까울수록 관계가 없음

### ① 데이터 전처리★

In [76]:
df = kospi.copy()
df['kosdaq'] = kosdaq['closePrice']
df['usd'] = usd['closePrice']
df = df.rename(columns = {'closePrice' : 'kospi'})

### ② 컬럼의 데이터 타입 변경 (str -> float)
* df[column].apply() : 모든 데이터를 함수에 대입한 결과를 출력

In [96]:
df['kospi'] = df['kospi'].apply(lambda data: float(data.replace(',','')))
df['kosdaq'] = df['kosdaq'].apply(lambda data: float(data.replace(',','')))
df['usd'] = df['usd'].apply(lambda data: float(data.replace(',','')))
df.dtypes

#추가 방법 : pd.to_numeric()

### ③ 피어슨 상관계수

In [74]:
df[['kospi','kosdaq','usd']].corr()

,kospi,kosdaq,usd
kospi,1.000000,0.984086,-0.878571
kosdaq,0.984086,1.000000,-0.821812
usd,-0.878571,-0.821812,1.000000


### ④ 결과
* kospi - kosdaq : 0.984 : 1과 가까우면 강한 양의 상관관계
* kospi - usd : -0.878 : -1과 가까우면 강한 음의 상관관계 (인사이트 : kospi 주식을 사는 것이 낫다)

----------------------------------------------------------------------------------------------------

## 7. 추가 학습
### 1) copy() : 값복사
* 배열, 데이터프레임일 때 사용 가능
* 2차원 이상의 배열에서 copy 쓰면 아마 내부에 리스트는 얕은 복사가 될 것
* 안의 라시트도 복사해주려면 deepcopy 써야될 것

In [97]:
data1 = [1, 2, 3] # [1,2,3]의 주소값을 data1이 갖게 함(참조시킴)

# 얕은복사(call by reference) : 주소값 복사
data2 = data1 # data1의 주소값 -> data2

# 깊은복사(call by value) : 값복사
data3 = data1.copy() # [1,2,3] 값을 만들고, 주소값을 새로 만든 후 새 주소값을 data3가 갖게 함. 
print(data1, data2, data3) # [1, 2, 3], [1, 2, 3], [1, 2, 3]

data1[1] = 4 # data1, data2가 갖는 메모리 주소가 같기에 둘다 바뀜 (원본 손상)
print(data1, data2, data3) # [1, 4, 3], [1, 4, 3], [1, 2, 3]

### 2) apply(func) : 모든 데이터를 func 적용시킨 결과 출력

In [98]:
# 데이터프레임 생성
df = pd.DataFrame([{'age':23}, {'age':36}, {'age': 27}])

# 연령대 변환 함수 생성
def change_ages(age):
    return age // 10 * 10

# 연령대 컬럼 추가
df['ages'] = df['age'].apply(change_ages)
df

### 3) lambda : 일회성 함수
* 파라미터 받아서 바로 리턴
* argument에서 바로 사용하는 용도로 쓰임
* 사용 이유 : 메모리 절약 및 빠른 속도
* lambda 사용법 : **lambda 파라미터: return 값**

In [99]:
# 함수 3개 생성 (= 메모리 3칸 사용)
def plus(n1, n2):
    return n1 + n2

def minus(n1, n2):
    return n1 - n2

def calc(func, n1, n2):
    return func(n1, n2)

# 생성한 함수 vs lambda
print(calc(plus, 1, 2), calc(minus, 1, 2))
print(calc(lambda n1, n2: n1 + n2, 1, 2), calc(lambda n1, n2:n1 - n2, 1, 2))

# 생성한 함수 vs lambda
plus_lambda = lambda n1, n2: n1 + n2
print(plus(2, 3), plus_lambda(2, 3))

---------
---------

## 8. 요약
### 1) 웹페이지 종류
- 정적페이지 : 데이터가 변경될때 URL 변경 O : HTML
- 동적페이지 : 데이터가 변경될때 URL 변경 X : JSON

### 2) 웹크롤링 절차
    ① 웹서비스 분석(URL) : 크롬 개발자 도구 사용<br>
    ② requests(url) > response(json) : JSON(str) 형태로 수집
        - 정적페이지 경우, response(html)
    ③ JSON(str) > list, dict > DataFrame : 형태 변환

### 3) 데이터 분석
    ① 데이터 전처리  
    ② 상관관계분석 : 피어슨 상관계수

----

# Ⅱ API
- Application Programing Interface
- API 를 사용해서 데이터를 수집하는 것은 서비스에 데이터를 제공하는 공식적인 방법으로 데이터 수집

## 1. API 절차
1. APP 등록
2. APP Key 받아오기 (유저 판별을 위해)
3. URL에 APP Key를 넣기
4. 데이터 보내기 : request -> response(JSON)

## 2. Naver Rest API : papago api : 번역 서비스
* 파파고 번역 api
* 통합검색어 트렌드 api

In [4]:
import requests, json
import pandas as pd

### ① request token 얻기
1. https://developers.naver.com 접속 후 로그인
2. Request Token 얻기 : 애플리케이션등록 -> app_key 획득 (앱키 없으면 패스워드 없이 보내는 것, 거부당함)
3. app_key를 이용해서 데이터 가져오기

In [8]:
CLIENT_ID, CLIENT_SECRET = "비", "밀"

### ② Naver API Document 확인 > URL
- Documents > Papago 번역 > API 레퍼런스 > 요청 URL 복사

In [9]:
txt = '파이썬은 재미있다.'
url = 'https://openapi.naver.com/v1/papago/n2mt'

# url 파라미터 설정
params = {'source' : 'ko', 'target' : 'en', 'text' : txt}

# app_key 값은 header에 저장 (웹페이지의 요청 예에서 -H는 header의 약자, -D는 data의 약자)
headers = {
    'Content-Type' : 'application/json', 
    'X-Naver-Client-Id' : CLIENT_ID,
    'X-Naver-Client-Secret' : CLIENT_SECRET
}

### ③ request(url, app_key) > response(json) : JSON(str)
- json.dumps()
    - 인터넷 트래픽에서는 영문, 숫자, 특수문자만 사용 가능
    - 한글과 같은 문자를 영문, 숫자, 특수문자로 인코딩

In [50]:
response = requests.post(url, json.dumps(params), headers = headers)
response

In [49]:
response.text

### ④ JSON(str) > list, dict > str 
- 데이터가 여러 개가 아니므로 DataFrame 아닌 str로 변환

In [46]:
ko_to_en = response.json()['message']['result']['translatedText'] # dict 형태로 변환
ko_to_en

### ⑤ 함수로 만들기

In [33]:
def translate(txt):
    CLIENT_ID, CLIENT_SECRET = "비", "밀"
    url = 'https://openapi.naver.com/v1/papago/n2mt'
    params = {'source' : 'ko', 'target' : 'en', 'text' : txt}
    headers = {
        'Content-Type' : 'application/json', 
        'X-Naver-Client-Id' : CLIENT_ID, 
        'X-Naver-Client-Secret' : CLIENT_SECRET
    }
    response = requests.post(url, json.dumps(params), headers = headers)
    
    return response.json()['message']['result']['translatedText']

# txt = '웹크롤링은 재밌다.'
# ko_to_en = translate(txt)
# ko_to_en

### ⑥ 한글 Excel 파일을 영문 Excel 파일로 변경

In [47]:
%ls # 주피터 노트북에서 제공하는 기능 : 현재 디렉토리에서의 파일 보여줌

In [38]:
covid = pd.read_excel('covid.xlsx')[['category', 'title']]
covid['title_en'] = covid['title'].apply(translate)

# utf-8-sig : excel 에서 사용하는 인코딩 방식과 호환되는 utf-8 인코딩 방식
covid.to_excel('covid_en.xlsx', index = False, encoding = 'utf-8-sig')

## 3. 추가 학습
### 1) 인코딩 
- encoding : 'a' -> 0001
- decoding : 0001 -> 'a'
- decoding 할 때, encoding 방식과 동일하게 해야 '쀅쀅?<>' 이런 이상한 문자가 나오지 않음

- ascii(한글:4byte) : 영문, 숫자, 특수문자 표현
- euckr(한글:8byte)
- utf8(한글:16byte) : 모든 나라의 언어를 표현(지금은 하드웨어 발전으로 utf8을 사용하면 됨)
    - utf-8-sig : sig를 붙혀야 엑셀에서 글자가 깨지지 않음

### 2) 함수 : Scope
- 변수 선언할 때 함수 밖에서 선언하는 것과 함수 안에서 선언하는 변수의 메모리 저장공간이 다름
- 전역(global)영역 : 함수 밖의 영역
- 지역(local)영역 : 함수 안의 영역
- global : 지역영역에서 전역영역의 변수를 가져올 때의 명령어

In [48]:
data = 10

def change1():
    data = 20

change1()
print(data) # 10

def change2():
    global data
    data = 20

change2()
print(data) # 20

# 4. 실습과제
- https://finance.daum.net/exchanges
- headers : referer, user-agent 설정